# AI in Ecology: A Domain Knowledge Centred XAI Framework

## Species Distribution Modelling: Tigers in India

### Overview
The SDM framework pdf flow diagram adheres to the standard structure of a software development lifecycle (SDLC), but it pays additional emphasis to stakeholder responsibilities and the leveraging of domain knowledge to ensure model interpretability. Each step in the flow diagram falls within the below sections:

1. Requirements gathering
2. Data collection
3. Data processing
4. Model development
5. Model evaluation
6. Model interpretation
7. Model deployment

A key difference between a standard SDLC and the proposed SDM framework is the decision nodes in the diagram. These nodes ensure that the requirements are being met throughout by using an iterative development approach.

This notebook accompanies the pdf framework by providing practical coding approaches that could be mirrored in a real-world SDM project. Whereas the pdf uses colour to highlight relevant stakeholders, this notebook will explicitly state the relevant stakeholders at each stage. In this notebook, I am acting as the data scientist. An obvious limitation is that this demonstration does not utilise a project manager or a domain expert. I will account for this by referring to where input from these stakeholders would be useful.

The example this notebook explores is modelling the distribution of tigers in India. The modelling approach I will use in Maximum Entropy (MaxEnt), which will be described in more detail later in the notebook.

### Requirements Gathering
1. Establish stakeholder contacts and communication - Project Manager
2. Define monetary and time constraints - Project Manager
3. Establish clear model performance and interpretability requirements - Domain Expert
4. Establish the data collection requirements - Domain Expert

**Comments**

In this example. I used arbitrary data requirements which were to gather tiger occurrence data from 2019 to 2023 in India, and to gather environmental data for the same timeframe and location. A real-world example would involve completing steps 1-3 before deciding on the specifics of the collected data. Establishing communication channels is vital to ensure a range of ideas are explored that can then be measured up against the agreed monetary and time constraints. Capturing new data may be time consuming and expensive, and if there is a limited budget or time window, there may be a heavier reliance on existing data. In terms of performance and interpretability requirements, a primary question is to determine if predictive power or causal inference should be prioritised. This is up to the domain expert, as they will be the end-users once the model is deployed.

### Data Collection
1. Search for existing available data - Data Scientist
2. Capture new species/environmental data - Domain Expert
3. Check that the data conforms to spatial and temporal requirements - Domain Expert

**Comments**
For this walkthrough, existing peer reviewed data sources were used to gather species occurrence and environmental data. No new data was collected for this example. Details of how the data was collected are below:

#### Data source information
**Tiger species occurence data:**
This data was sourced from the Global Biodiversity Information Facility: https://www.gbif.org. To obtain the tiger_occurrence_data.csv file, I used the "occurrences" page and then used the below filter options:
* Country or area: India
* Scientific name: Panthera tigris (Linnaeus, 1758)
* Year: Between 2019 and 2023
* Months: All months
* Citation: GBIF.org (3 September 2024) GBIF Occurrence Download https://doi.org/10.15468/dl.sgeq2r

For this example, the data was isolated to India and years 2019-2023. However, the filters can easily be used in real use-cases to extract species occurrence data relevant to the project requirements.

**Environmental data**
This data was sourced from the Climate Data Store (CDS): https://cds-beta.climate.copernicus.eu. To obtain the ERA5_environmental_data.nc file, I selected "ERA5-Land monthly averaged data from 1950 to present" as this dataset allowed me to extract land environmental features filtered to the same spatio and temporal range as the tiger species occurrence data. The data request criteria is below:
* Product type: Monthly averaged reanalysis by hour of day
* Variables: 2m dewpoint temperature, 2m temperature, Surface solar radiation downwards, Runoff, Total evaporation, 10m u-component of wind, 10m v-component of wind, Total precipitation, Leaf area index, high vegetation
* Year: 2019-2023
* Month: All months
* Time: 12:00pm
* Geographical area: North: 37°, West: 68°, South: 8°, East: 97° (India)
* Data format: GRIB
* Citation: Muñoz Sabater, J. (2019): ERA5-Land monthly averaged data from 1950 to present. Copernicus Climate Change Service (C3S) Climate Data Store (CDS). DOI: 10.24381/cds.68d2bb30 (Accessed on 3 September 2024)

### Data Processing
1. Perform data cleaning (data scientist) and avoid dropping ecologically relevant data with support from a ecology domain expert
2. Feature engineering and selection (data scientist) with support from an ecology domain expert to select and derive ecologically relevant features.

**Comments**
* This section focuses on processing the collected data for use in MaxEnt. 
* After loading the data into the notebook, I will pre-process both datasets so they are ready for use in MaxEnt
* As this example uses MaxEnt, the environmental data needs to be convered to ASCII format for compatability with the MaxEnt software

In [1]:
# Install the project requirements
# !pip install -r requirements.txt

In [2]:
# Import libraries needed to load in the tiger occurrence and ERA5 environmental data
import pandas as pd
import os
import xarray as xr

# Get the current working directory
cwd = os.getcwd()

# Load in the species occurrence data from the dataset folder
occurrence_df = pd.read_csv(os.path.join(cwd, 'datasets/tiger_occurrence_data.csv'), sep='\t')

# Load in the environmental data
nc_address = os.path.join(cwd, 'datasets/ERA5_environmental_data.nc')
env_data = xr.open_dataset(nc_address, engine='netcdf4')

In [3]:
# Function to automate the initial investigation of the imported occurrence dataset

def investigate_data(data, name):
    """
    Description:
    This function returns the shape, column names, data types, number of unique values, and checks for missing values.
    
    Args:
        data (DataFrame): The dataset to investigate
        name (string): The name of the dataset

    Returns:
        DataFrame: Returns the top 5 rows of the dataset
    """
    
    # Get the shape of the data
    print(f"The {name} dataset has {data.shape[0]} rows and {data.shape[1]} columns.\n")
    
    # Get the column names
    print(f"The columns in the {name} dataset are: {data.columns}\n")
    
    # Get the data types of each column
    print(f"The data types of each column in the {name} dataset are:")
    print(data.dtypes.groupby(data.dtypes).size())
    
    # Get the number of unique values in each column
    print(f"\nThe number of unique values in each column of the {name} dataset are:")
    print(data.nunique().tolist())
    
    # Check if there any missing values in the dataset
    print(f"There are missing values in the dataset? {data.isnull().values.any()}\n")
    
    # Show the first 5 rows of the dataset
    return data.head()
    
# Investigate the tiger occurrence data
investigate_data(occurrence_df, "Tiger Occurrence")

The Tiger Occurrence dataset has 2545 rows and 50 columns.

The columns in the Tiger Occurrence dataset are: Index(['gbifID', 'datasetKey', 'occurrenceID', 'kingdom', 'phylum', 'class',
       'order', 'family', 'genus', 'species', 'infraspecificEpithet',
       'taxonRank', 'scientificName', 'verbatimScientificName',
       'verbatimScientificNameAuthorship', 'countryCode', 'locality',
       'stateProvince', 'occurrenceStatus', 'individualCount',
       'publishingOrgKey', 'decimalLatitude', 'decimalLongitude',
       'coordinateUncertaintyInMeters', 'coordinatePrecision', 'elevation',
       'elevationAccuracy', 'depth', 'depthAccuracy', 'eventDate', 'day',
       'month', 'year', 'taxonKey', 'speciesKey', 'basisOfRecord',
       'institutionCode', 'collectionCode', 'catalogNumber', 'recordNumber',
       'identifiedBy', 'dateIdentified', 'license', 'rightsHolder',
       'recordedBy', 'typeStatus', 'establishmentMeans', 'lastInterpreted',
       'mediaType', 'issue'],
      dtype='

,gbifID,datasetKey,occurrenceID,kingdom,phylum,class,order,family,genus,species,...,identifiedBy,dateIdentified,license,rightsHolder,recordedBy,typeStatus,establishmentMeans,lastInterpreted,mediaType,issue
0,4926309339,50c9509d-22c7-4a22-a47d-8c48425ef4a7,https://www.inaturalist.org/observations/23606...,Animalia,Chordata,Mammalia,Carnivora,Felidae,Panthera,Panthera tigris,...,Manav,2024-08-16T11:52:44,CC_BY_NC_4_0,Manav,Manav,NaN,NaN,2024-08-29T05:41:34.529Z,StillImage,COORDINATE_ROUNDED;CONTINENT_DERIVED_FROM_COOR...
1,4926056602,50c9509d-22c7-4a22-a47d-8c48425ef4a7,https://www.inaturalist.org/observations/23606...,Animalia,Chordata,Mammalia,Carnivora,Felidae,Panthera,Panthera tigris,...,Manav,2024-08-16T11:46:55,CC_BY_NC_4_0,Manav,Manav,NaN,NaN,2024-08-29T05:41:27.926Z,StillImage,COORDINATE_ROUNDED;CONTINENT_DERIVED_FROM_COOR...
2,4908758130,50c9509d-22c7-4a22-a47d-8c48425ef4a7,https://www.inaturalist.org/observations/22751...,Animalia,Chordata,Mammalia,Carnivora,Felidae,Panthera,Panthera tigris,...,Cyan Fox,2024-07-16T09:24:26,CC_BY_NC_4_0,David Phinehas,David Phinehas,NaN,NaN,2024-08-29T06:18:07.509Z,StillImage,COORDINATE_ROUNDED;CONTINENT_DERIVED_FROM_COOR...
3,4901747602,50c9509d-22c7-4a22-a47d-8c48425ef4a7,https://www.inaturalist.org/observations/20684...,Animalia,Chordata,Mammalia,Carnivora,Felidae,Panthera,Panthera tigris,...,Darrell Parsons,2024-04-12T21:16:35,CC_BY_NC_4_0,Darrell Parsons,Darrell Parsons,NaN,NaN,2024-08-29T05:34:24.549Z,StillImage,COORDINATE_ROUNDED;CONTINENT_DERIVED_FROM_COOR...
4,4901689794,50c9509d-22c7-4a22-a47d-8c48425ef4a7,https://www.inaturalist.org/observations/15138...,Animalia,Chordata,Mammalia,Carnivora,Felidae,Panthera,Panthera tigris,...,Gabi Rusu,2023-03-17T14:23:52,CC_BY_4_0,Gabi Rusu,Gabi Rusu,NaN,NaN,2024-08-29T05:19:48.132Z,StillImage,COORDINATE_ROUNDED;CONTINENT_DERIVED_FROM_COOR...


In [4]:
# Drop all columns that aren't needed for presence only species distribution modeling
occ_cols = ['species','decimalLongitude', 'decimalLatitude']
tiger_occurrences = occurrence_df[occ_cols]

# Rename coorindate columns for compatibilitty with MaxEnt
tiger_occurrences.rename(columns={'decimalLatitude':'latitude', 'decimalLongitude':'longitude'}, inplace=True)

# Recheck for missing values in the tiger occurrence data after dropping columns
print(f"The number of records containing missing values in the tiger occurrence data are: \n{tiger_occurrences.isnull().sum()}\n")

The number of records containing missing values in the tiger occurrence data are: 
species       0
longitude    10
latitude     10
dtype: int64



/var/folders/wz/1k6yj9pn3kj1s5gppbqtnn280000gn/T/ipykernel_77836/3972411896.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tiger_occurrences.rename(columns={'decimalLatitude':'latitude', 'decimalLongitude':'longitude'}, inplace=True)


In [5]:
# Drop the records containing missing coordinates
tiger_occurrences.dropna(subset=['latitude', 'longitude'], inplace=True)

# Rename the species column so there is no space in the species name
tiger_occurrences['species'] = tiger_occurrences['species'].str.replace('Panthera tigris', 'pantheraTigris')

# Recheck for missing values
print(f"Are there any missing values left in the tiger occurrence dataset? {tiger_occurrences.isnull().values.any()}")

# Check top 5 rows of the tiger occurrence data
tiger_occurrences.head()

Are there any missing values left in the tiger occurrence dataset? False


/var/folders/wz/1k6yj9pn3kj1s5gppbqtnn280000gn/T/ipykernel_77836/1903208550.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tiger_occurrences.dropna(subset=['latitude', 'longitude'], inplace=True)
/var/folders/wz/1k6yj9pn3kj1s5gppbqtnn280000gn/T/ipykernel_77836/1903208550.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tiger_occurrences['species'] = tiger_occurrences['species'].str.replace('Panthera tigris', 'pantheraTigris')


,species,longitude,latitude
0,pantheraTigris,78.583812,29.734195
1,pantheraTigris,78.425719,29.700184
2,pantheraTigris,78.352840,30.122850
3,pantheraTigris,79.064665,29.556724
4,pantheraTigris,81.068834,23.739201


In [6]:
# Now perform initial investigation of the environmental data
env_data

<xarray.Dataset> Size: 183MB
Dimensions:     (valid_time: 60, latitude: 291, longitude: 291)
Coordinates:
    number      int64 8B ...
  * valid_time  (valid_time) datetime64[ns] 480B 2019-01-01 ... 2023-12-01
  * latitude    (latitude) float64 2kB 37.0 36.9 36.8 36.7 ... 8.3 8.2 8.1 8.0
  * longitude   (longitude) float64 2kB 68.0 68.1 68.2 68.3 ... 96.8 96.9 97.0
    expver      (valid_time) <U4 960B ...
Data variables:
    d2m         (valid_time, latitude, longitude) float32 20MB ...
    t2m         (valid_time, latitude, longitude) float32 20MB ...
    ssrd        (valid_time, latitude, longitude) float32 20MB ...
    ro          (valid_time, latitude, longitude) float32 20MB ...
    e           (valid_time, latitude, longitude) float32 20MB ...
    u10         (valid_time, latitude, longitude) float32 20MB ...
    v10         (valid_time, latitude, longitude) float32 20MB ...
    tp          (valid_time, latitude, longitude) float32 20MB ...
    lai_hv      (valid_time, latitude, longitude) float32 20MB ...
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2024-09-03T18:13 GRIB to CDM+CF via cfgrib-0.9.1...

* There are 60 time points relating to each month in the period 2019 - 2023
* There are 291 latitude and longitude points
* The total number of values for each Data variable is therefore 60 * 291 * 291 = 5,080,060

In [7]:
# As latitude and longitude are 1 decimal place, round the tiger occurrence data to 1 decimal place for spatial consistency
tiger_occurrences['latitude'] = tiger_occurrences['latitude'].round(1)
tiger_occurrences['longitude'] = tiger_occurrences['longitude'].round(1)

# The tiger occurrence data is now ready for use. Export the clean version as a csv file.
tiger_occurrences.to_csv(os.path.join(cwd, 'samples/clean_tiger_occurrences.csv'), index=False)

/var/folders/wz/1k6yj9pn3kj1s5gppbqtnn280000gn/T/ipykernel_77836/3942994896.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tiger_occurrences['latitude'] = tiger_occurrences['latitude'].round(1)
/var/folders/wz/1k6yj9pn3kj1s5gppbqtnn280000gn/T/ipykernel_77836/3942994896.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tiger_occurrences['longitude'] = tiger_occurrences['longitude'].round(1)


In [8]:
# Check for null values in the environmental data
env_data.isnull().sum()

<xarray.Dataset> Size: 80B
Dimensions:  ()
Coordinates:
    number   int64 8B ...
Data variables:
    d2m      int64 8B 1684860
    t2m      int64 8B 1684860
    ssrd     int64 8B 1684860
    ro       int64 8B 1684860
    e        int64 8B 1684860
    u10      int64 8B 1684860
    v10      int64 8B 1684860
    tp       int64 8B 1684860
    lai_hv   int64 8B 1684860

These null values will be addressed later on when creating ASCII files for MaxEnt

#### Domain-Based Feature Engineering
* Exploiting ecological knowledge from domain experts during feature engineering is referenced in the pdf framework. 
* A common feature engineering method is to multiply features to create interactions. A simple function will be provided to implement this.
* For this example, researching the ERA5 environmental data provided me with formulae to derive wind speed from the u and v components of wind (ECMWF, 2024)

In [9]:
# Function to multiply Data variables from the environmental data to create interaction terms
def multiply_features(data, feature1, feature2):
    """
    Description:
    This function multiplies two features in a dataset to create a new interaction feature.
    
    Args:
        data (xarray.Dataset): The dataset containing the features to multiply
        feature1 (string): The first feature to multiply
        feature2 (string): The second feature to multiply

    Returns:
        xarray.Dataset: Returns the dataset with the new interaction term
    """
    
    # Multiply the two features together
    data[f'{feature1}_{feature2}'] = data[feature1] * data[feature2]
    
    return data

In [10]:
# To create a wind speed feature, we need to import numpy for the mathematical operations
import numpy as np

# Function to derive wind speed
def wind_speed(data):
    """
    Description: This function calculates the wind speed from the u and v wind components.
    
    Args:
        data (xarray.Dataset): The dataset containing the u and v wind components
     
     Returns:
        xarray.Dataset: Returns the dataset with the new wind speed feature   
    """
    
    # Calculate the wind speed using the formula from (ECMWF, 2024)
    data['wind_speed'] = np.sqrt(data['u10']**2 + data['v10']**2)
    
    return data

# Create the wind speed feature for env_data
env_data = wind_speed(env_data)

# Drop the u and v wind components. Wind speed can be used in their place. 
# In a real-world scenario, the domain expert would determine which features need to be created or dropped.
env_data = env_data.drop_vars(['u10', 'v10'])

In [11]:
# Check filled data
env_data

<xarray.Dataset> Size: 163MB
Dimensions:     (valid_time: 60, latitude: 291, longitude: 291)
Coordinates:
    number      int64 8B ...
  * valid_time  (valid_time) datetime64[ns] 480B 2019-01-01 ... 2023-12-01
  * latitude    (latitude) float64 2kB 37.0 36.9 36.8 36.7 ... 8.3 8.2 8.1 8.0
  * longitude   (longitude) float64 2kB 68.0 68.1 68.2 68.3 ... 96.8 96.9 97.0
    expver      (valid_time) <U4 960B ...
Data variables:
    d2m         (valid_time, latitude, longitude) float32 20MB 275.1 ... nan
    t2m         (valid_time, latitude, longitude) float32 20MB 283.5 ... nan
    ssrd        (valid_time, latitude, longitude) float32 20MB 8.858e+06 ... nan
    ro          (valid_time, latitude, longitude) float32 20MB 5.946e-06 ... nan
    e           (valid_time, latitude, longitude) float32 20MB -0.0006349 ......
    tp          (valid_time, latitude, longitude) float32 20MB 0.0009384 ... nan
    lai_hv      (valid_time, latitude, longitude) float32 20MB 0.3915 ... nan
    wind_speed  (valid_time, latitude, longitude) float32 20MB 0.7098 ... nan
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2024-09-03T18:13 GRIB to CDM+CF via cfgrib-0.9.1...

In [ ]:
# OPTIONAL: Save updated environmental data to a new netCDF file
#net_cdf_address = os.path.join(cwd, 'datasets/clean_ERA5_environmental_data.nc')
#env_data_filled.to_netcdf(net_cdf_address)

* Now the environmental data is clean and some feature engineering has been completed. The clean netCDF data needs to be converted to ASCII format for use in MaxEnt.
* To do this, first convert the clean netCDF data to a pandas DataFrame, then create .asc files for the environmental variables

In [12]:
# Convert the scaled environmental data to a pandas DataFrame
env_data = env_data.to_dataframe().reset_index()

# Investigate the scaled environmental
investigate_data(env_data, "Environmental Data")

The Environmental Data dataset has 5080860 rows and 13 columns.

The columns in the Environmental Data dataset are: Index(['valid_time', 'latitude', 'longitude', 'number', 'expver', 'd2m', 't2m',
       'ssrd', 'ro', 'e', 'tp', 'lai_hv', 'wind_speed'],
      dtype='object')

The data types of each column in the Environmental Data dataset are:
datetime64[ns]    1
int64             1
float32           8
float64           2
object            1
dtype: int64

The number of unique values in each column of the Environmental Data dataset are:
[60, 291, 291, 1, 1, 786226, 609379, 2810611, 799865, 2729021, 2919514, 47087, 3216808]
There are missing values in the dataset? True



,valid_time,latitude,longitude,number,expver,d2m,t2m,ssrd,ro,e,tp,lai_hv,wind_speed
0,2019-01-01,37.0,68.0,0,0001,275.119812,283.503845,8857572.0,0.000006,-0.000635,0.000938,0.391479,0.709843
1,2019-01-01,37.0,68.1,0,0001,275.483093,283.665955,8853786.0,0.000005,-0.000696,0.000969,0.000000,0.602009
2,2019-01-01,37.0,68.2,0,0001,276.069031,283.732361,8851458.0,0.000005,-0.000756,0.001016,0.000000,0.615537
3,2019-01-01,37.0,68.3,0,0001,276.092468,283.650330,8849920.0,0.000006,-0.000769,0.001059,0.477905,0.703381
4,2019-01-01,37.0,68.4,0,0001,275.969421,283.427673,8851104.0,0.000006,-0.000789,0.001091,2.445801,0.727230


In [14]:
# Rename columns based on the data from the "ERA5_environmental_data" data source so they are more informative
column_name_dict = {
    "d2m" : "2m_dewpoint_temperature",
    "t2m" : "2m_temperature",
    "ssrd" : "surface_solar_radiation_downwards",
    "ro" : "runoff",
    "e" : "evaporation",
    "u10" : "10m_u_component_of_wind",
    "v10" : "10m_v_component_of_wind",
    "tp" : "total_precipitation",
    "lai_hv" : "leaf_area_index_high_vegetation",
}

env_df = env_data.rename(columns=column_name_dict)

# Isolate environmental data columns that will be used in MaxEnt
non_env_vars = ['valid_time', 'latitude', 'longitude', 'number', 'expver']
env_vars = [col for col in env_df.columns if col not in non_env_vars]

env_df[env_vars]

,2m_dewpoint_temperature,2m_temperature,surface_solar_radiation_downwards,runoff,evaporation,total_precipitation,leaf_area_index_high_vegetation,wind_speed
0,275.119812,283.503845,8857572.0,0.000006,-0.000635,0.000938,0.391479,0.709843
1,275.483093,283.665955,8853786.0,0.000005,-0.000696,0.000969,0.000000,0.602009
2,276.069031,283.732361,8851458.0,0.000005,-0.000756,0.001016,0.000000,0.615537
3,276.092468,283.650330,8849920.0,0.000006,-0.000769,0.001059,0.477905,0.703381
4,275.969421,283.427673,8851104.0,0.000006,-0.000789,0.001091,2.445801,0.727230
...,...,...,...,...,...,...,...,...
5080855,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5080856,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5080857,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5080858,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


* The environmental variables need to be scaled before converting to ASCII files due to the large range
* The NaN values can be replaced with -9999, so MaxEnt knows these are null values. However, scaling must be done first so the scaler doesn't scale the -9999 values.

In [15]:
# Import StandardScaler from the sklearn library
from sklearn.preprocessing import StandardScaler

# Standardize the environmental data
scaler = StandardScaler()
env_df[env_vars] = scaler.fit_transform(env_df[env_vars])

# Chech new standardized environmental data
env_df.describe().T # Transpose the data for better readability

,count,mean,min,25%,50%,75%,max,std
valid_time,5080860,2021-06-16 06:24:00.000003072,2019-01-01 00:00:00,2020-03-24 06:00:00,2021-06-16 00:00:00,2022-09-08 12:00:00,2023-12-01 00:00:00,NaN
latitude,5080860.0,22.5,8.0,15.2,22.5,29.8,37.0,8.400398
longitude,5080860.0,82.5,68.0,75.2,82.5,89.8,97.0,8.400398
number,5080860.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2m_dewpoint_temperature,3396000.0,-0.0,-2.844349,-0.637541,0.268676,0.847299,1.384008,1.0
2m_temperature,3396000.0,0.0,-3.175597,-0.749994,0.410736,0.731989,1.700797,1.0
surface_solar_radiation_downwards,3396000.0,-0.0,-2.750938,-0.794356,-0.138862,0.801438,2.814886,1.0
runoff,3396000.0,0.0,-0.395204,-0.385675,-0.337691,-0.126635,23.355539,1.0
evaporation,3396000.0,0.0,-3.383628,-0.842586,0.051407,0.919417,2.037482,1.0
total_precipitation,3396000.0,-0.0,-0.664663,-0.609028,-0.421816,0.186847,12.263767,1.0


* Environmental variables now have a mean of 0 and a standard deviation of 1
* These variables can now be converted into .asc files for use in MaxEnt. But first, replace the null values with -9999

In [17]:
env_df_filled = env_df.fillna(-9999)

In [32]:
env_df_filled.describe().T

,count,mean,min,25%,50%,75%,max,std
valid_time,5080860,2021-06-16 06:24:00.000003072,2019-01-01 00:00:00,2020-03-24 06:00:00,2021-06-16 00:00:00,2022-09-08 12:00:00,2023-12-01 00:00:00,NaN
latitude,5080860.0,22.5,8.0,15.2,22.5,29.8,37.0,8.400398
longitude,5080860.0,82.5,68.0,75.2,82.5,89.8,97.0,8.400398
number,5080860.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2m_dewpoint_temperature,5080860.0,-3315.761963,-9999.0,-9999.0,-0.62705,0.527462,1.384008,4707.443848
2m_temperature,5080860.0,-3315.761719,-9999.0,-9999.0,-0.740664,0.592859,1.700797,4707.443848
surface_solar_radiation_downwards,5080860.0,-3315.759277,-9999.0,-9999.0,-0.789779,0.302061,2.814886,4707.443848
runoff,5080860.0,-3315.761963,-9999.0,-9999.0,-0.385501,-0.274044,23.355539,4707.443848
evaporation,5080860.0,-3315.759033,-9999.0,-9999.0,-0.835872,0.464589,2.037482,4707.443848
total_precipitation,5080860.0,-3315.760986,-9999.0,-9999.0,-0.608206,-0.201544,12.263767,4707.443848


In [49]:
# Additional imports to create asc files from the environmental data
from scipy.interpolate import griddata
import rasterio
from rasterio.transform import from_origin

# Define the latitude and longitude grid
lat_min = np.floor(env_df_filled['latitude'].min() * 10) / 10  # Round down to nearest 0.1
lat_max = np.ceil(env_df_filled['latitude'].max() * 10) / 10   # Round up to nearest 0.1
lon_min = np.floor(env_df_filled['longitude'].min() * 10) / 10
lon_max = np.ceil(env_df_filled['longitude'].max() * 10) / 10

# Define the resolution of the grid as 0.1 as coordinates are rounded to 1 decimal place. The grid steps from the minimum to maximum latitude and longitude in 0.1 increments
grid_lat_res, grid_lon_res = np.mgrid[lat_min:lat_max:0.1, lon_min:lon_max:0.1]

# Create an empty dictionary to store the environmental grid data
env_grid = {}

# Loop through variables in the env_var list and interpolate the data to the dictionary
for var in env_vars:
    grid_data = griddata(
        points=(env_df_filled['longitude'], env_df_filled['latitude']), # Grid coordinates to interpolate
        values=env_df_filled[var], # Environmental variable to interpolate
        xi=(grid_lon_res, grid_lat_res), # Resolution of the grid
        method='nearest' # Use nearest neighbour method for interpolation to avoid creating null values
    )
    
    # Store the above grid data in the dictionary
    env_grid[var] = grid_data
    
    # Transform the data from rows and columns to geo-coordinates using the rasterio library
    transform = from_origin(lon_min, lat_max, 0.1, 0.1) # 0.1 defines the height and width of each pixel in the grid
    
    # Store metadata for the new .asc file
    meta = {
        'driver': 'AAIGrid',
        'height': grid_data.shape[0],
        'width': grid_data.shape[1],
        'count': 1,
        'dtype': 'float64',
        'crs': 'EPSG:4326', # This is the coordinate reference system for latitude and longitude coordinates
        'transform': transform
    }
    
    # Write the grid data to a .asc file to the layers folder, as per the MaxEnt tutorial (Phillips, 2017)
    with rasterio.open(f'layers/{var}.asc', 'w', **meta, nodata=-9999) as dst:
        dst.write(grid_data, 1)
        
    # List the asc files created and where they are saved to
    print(f"Created {var}.asc file in the layers folder")

Created 2m_dewpoint_temperature.asc file in the layers folder
Created 2m_temperature.asc file in the layers folder
Created surface_solar_radiation_downwards.asc file in the layers folder
Created runoff.asc file in the layers folder
Created evaporation.asc file in the layers folder
Created total_precipitation.asc file in the layers folder
Created leaf_area_index_high_vegetation.asc file in the layers folder
Created wind_speed.asc file in the layers folder


# Function to read .asc file and check for completeness
def check_asc_file(file_path):
    ncols, nrows = None, None
    grid_values = []

    with open(file_path, 'r') as file:
        lines = file.readlines()
        
        # Read header to get ncols and nrows
        for line in lines:
            if line.startswith('ncols'):
                ncols = int(line.split()[1])
            elif line.startswith('nrows'):
                nrows = int(line.split()[1])
            elif line.strip().replace('.', '', 1).isdigit() or "-" in line:
                # Collect all lines that represent grid values
                grid_values.extend(line.split())

    # Calculate the expected number of values
    if ncols and nrows:
        expected_values_count = ncols * nrows
        actual_values_count = len(grid_values)
        print(f"Expected number of values: {expected_values_count}")
        print(f"Actual number of values: {actual_values_count}")
        
        # Check if the actual values match the expected count
        if actual_values_count == expected_values_count:
            print("The file contains the correct number of values.")
        else:
            print("The file is incomplete or contains extra values.")
    else:
        print("Failed to read ncols or nrows from the file.")

# Provide the path to your .asc file
asc_file_path = os.path.join(layers_folder, '2m_temperature.asc')
check_asc_file(asc_file_path)